In [1]:
from datetime import datetime, timedelta, timezone
import requests
base_url = "http://127.0.0.1:8811"

Regime detection using ohlcv and vix data.

In [2]:
# get available tickers
response = requests.get(f"{base_url}/api/data-warehouse/tickers")
response.raise_for_status()
tickers = response.json()
tickers

[{'ticker': '3MINDIA', 'timeframes': []},
 {'ticker': 'ABB', 'timeframes': ['1d', '1h']},
 {'ticker': 'ADANIENT', 'timeframes': ['1d', '1h']},
 {'ticker': 'ADANIGREEN', 'timeframes': ['1d', '1h']},
 {'ticker': 'ADANIPORTS', 'timeframes': ['1d', '1h']},
 {'ticker': 'ADANIPOWER', 'timeframes': ['1d', '1h']},
 {'ticker': 'AMBUJACEM', 'timeframes': ['1d', '1h']},
 {'ticker': 'APOLLOHOSP', 'timeframes': ['1d', '1h']},
 {'ticker': 'ASIANPAINT', 'timeframes': ['1d', '1h']},
 {'ticker': 'AUROPHARMA', 'timeframes': ['1d', '1h']},
 {'ticker': 'AXISBANK', 'timeframes': ['1d', '1h']},
 {'ticker': 'BAJAJ-AUTO', 'timeframes': ['1d', '1h']},
 {'ticker': 'BAJAJFINSV', 'timeframes': ['1d', '1h']},
 {'ticker': 'BAJFINANCE', 'timeframes': ['1d', '1h']},
 {'ticker': 'BANKBARODA', 'timeframes': ['1d', '1h']},
 {'ticker': 'BEL', 'timeframes': ['1d', '1h']},
 {'ticker': 'BERGEPAINT', 'timeframes': ['1d', '1h']},
 {'ticker': 'BHARTIARTL', 'timeframes': ['1d', '1h']},
 {'ticker': 'BPCL', 'timeframes': ['1d', '

In [3]:
# Reusable fetch helper
def fetch_ohlcv_data(
    base_url: str,
    ticker: str,
    timeframe: str = "1h",
    years: int = 3,
    limit: int = 5000,
    offset: int = 0,
    timeout: int = 30,
    start_dt: datetime | None = None,
    end_dt: datetime | None = None,
) -> dict:
    end_dt = end_dt or datetime.now(timezone.utc)
    start_dt = start_dt or (end_dt - timedelta(days=365 * years))

    payload = {
        "ticker": ticker,
        "timeframe": timeframe,
        "range": {
            "start_epoch": int(start_dt.timestamp()),
            "end_epoch": int(end_dt.timestamp()),
        },
    }

    response = requests.post(
        f"{base_url}/api/data-warehouse/stocks/get",
        params={"limit": limit, "offset": offset},
        json=payload,
        timeout=timeout,
    )
    response.raise_for_status()
    ohlcv_data = response.json()

    return {
        "start_dt": start_dt,
        "end_dt": end_dt,
        "payload": payload,
        "response": response,
        "ohlcv_data": ohlcv_data,
    }



In [4]:
# fetch ohlcv data and vix data for the same period
ticker = "RELIANCE"
timeframe = "1h"
start_dt = datetime(2021, 1, 1, tzinfo=timezone.utc)
end_dt = datetime(2024, 1, 1, tzinfo=timezone.utc)
ohlcv_result = fetch_ohlcv_data(base_url, ticker, timeframe, start_dt=start_dt, end_dt=end_dt)
ohlcv_data = ohlcv_result["ohlcv_data"].get("candles", [])
vix_result = fetch_ohlcv_data(base_url, "INDIAVIX", timeframe, start_dt=ohlcv_result["start_dt"], end_dt=ohlcv_result["end_dt"])
vix_data = vix_result["ohlcv_data"].get("candles", [])
print(f"Fetched {len(ohlcv_data)} candles for {ticker} and {len(vix_data)} candles for VIX.")

Fetched 5000 candles for RELIANCE and 5000 candles for VIX.


## Feature Engineering (Regime Signals)
These features convert noisy 1h prices into more stationary regime-sensitive signals:
- **Log returns** for both asset and VIX
- **Volatility spread**: 20h realized asset vol minus 20h VIX vol proxy
- **Asset-VIX rolling correlation** (20h)
- **VIX velocity** (absolute and percent 1h change)

In [5]:
import numpy as np
import pandas as pd
import ta


def candles_to_close_df(
    candles: list,
    label: str,
    timestamp_candidates: tuple[str, ...] = (
        "timestamp",
        "ts",
        "time",
        "datetime",
        "date",
        "epoch",
    ),
    close_candidates: tuple[str, ...] = ("close", "Close", "c"),
) -> pd.DataFrame:
    if not candles:
        return pd.DataFrame(columns=["timestamp", f"{label}_close"], dtype="float64")

    # Normalize mixed candle objects into plain dict rows without relying on DataFrame(candles).
    rows: list[dict] = []
    for candle in candles:
        row = candle if isinstance(candle, dict) else None

        # Fallback: common list/tuple OHLCV layouts [ts, open, high, low, close, volume]
        if row is None and isinstance(candle, (list, tuple)) and len(candle) >= 5:
            row = {"timestamp": candle[0], "close": candle[4]}

        if row is None:
            continue

        ts_val = next((row.get(col) for col in timestamp_candidates if col in row), None)
        close_val = next((row.get(col) for col in close_candidates if col in row), None)

        if ts_val is None or close_val is None:
            continue

        rows.append({"timestamp": ts_val, f"{label}_close": close_val})

    out = pd.DataFrame.from_records(rows)
    if out.empty:
        return pd.DataFrame(columns=["timestamp", f"{label}_close"], dtype="float64")

    ts = out["timestamp"]
    if pd.api.types.is_numeric_dtype(ts):
        idx = pd.to_datetime(ts, unit="s", utc=True, errors="coerce")
        if idx.isna().all():
            idx = pd.to_datetime(ts, unit="ms", utc=True, errors="coerce")
    else:
        idx = pd.to_datetime(ts, utc=True, errors="coerce")

    out["timestamp"] = idx
    out[f"{label}_close"] = pd.to_numeric(out[f"{label}_close"], errors="coerce")
    out = out.dropna(subset=["timestamp", f"{label}_close"]).sort_values("timestamp")
    out = out.drop_duplicates(subset=["timestamp"], keep="last")
    return out


asset_df = candles_to_close_df(ohlcv_data, "asset")
vix_df = candles_to_close_df(vix_data, "vix")

if asset_df.empty or vix_df.empty:
    raise ValueError(
        "Asset/VIX candles are empty or not parseable. "
        "Re-run data fetch or inspect candle schema."
    )

# Align nearest hourly timestamps instead of strict exact matches.
features = pd.merge_asof(
    asset_df.sort_values("timestamp"),
    vix_df.sort_values("timestamp"),
    on="timestamp",
    direction="nearest",
    tolerance=pd.Timedelta("90min"),
)
features = features.dropna(subset=["asset_close", "vix_close"]).set_index("timestamp").sort_index()

# 1) Log returns (stationary-ish transformation)
features["asset_log_ret"] = np.log(features["asset_close"]).diff()
features["vix_log_ret"] = np.log(features["vix_close"]).diff()

# 2) Volatility spread (realized asset vol vs VIX vol proxy)
window = 20
features["asset_realized_vol_20h"] = features["asset_log_ret"].rolling(window).std()
features["vix_vol_proxy_20h"] = features["vix_log_ret"].rolling(window).std()
features["vol_spread_20h"] = features["asset_realized_vol_20h"] - features["vix_vol_proxy_20h"]

# 3) Asset-VIX rolling correlation
features["asset_vix_corr_20h"] = features["asset_log_ret"].rolling(window).corr(features["vix_log_ret"])

# 4) VIX velocity (absolute and percent hourly change)
features["vix_velocity_1h"] = features["vix_close"].diff()
features["vix_velocity_pct_1h"] = (features["vix_close"] / features["vix_close"].shift(1)) - 1

# 5) Technical indicators: RSI, MACD, EMA
# RSI (14-period Relative Strength Index)
features["rsi_14"] = ta.momentum.rsi(features["asset_close"], window=14)

# MACD (12, 26, 9)
macd_result = ta.trend.MACD(features["asset_close"], window_fast=12, window_slow=26, window_sign=9)
features["macd_line"] = macd_result.macd_diff()
features["macd_signal"] = macd_result.macd_signal()
features["macd_histogram"] = macd_result.macd_diff() - macd_result.macd_signal()

# EMA (12 and 26 period)
features["ema_12"] = ta.trend.ema_indicator(features["asset_close"], window=12)
features["ema_26"] = ta.trend.ema_indicator(features["asset_close"], window=26)

feature_cols = [
    "asset_close",
    "vix_close",
    "asset_log_ret",
    "vix_log_ret",
    "asset_realized_vol_20h",
    "vix_vol_proxy_20h",
    "vol_spread_20h",
    "asset_vix_corr_20h",
    "vix_velocity_1h",
    "vix_velocity_pct_1h",
    "rsi_14",
    "macd_line",
    "macd_signal",
    "macd_histogram",
    "ema_12",
    "ema_26",
]

print(f"Feature rows available: {len(features):,}")
features[feature_cols].tail(10)

Feature rows available: 4,961


,asset_close,vix_close,asset_log_ret,vix_log_ret,asset_realized_vol_20h,vix_vol_proxy_20h,vol_spread_20h,asset_vix_corr_20h,vix_velocity_1h,vix_velocity_pct_1h,rsi_14,macd_line,macd_signal,macd_histogram,ema_12,ema_26
timestamp,,,,,,,,,,,,,,,,
2023-12-28 07:45:00+00:00,1301.50,15.66,-0.002134,-0.009533,0.002658,0.025599,-0.022941,0.522307,-0.15,-0.009488,63.462181,0.426879,5.203232,-4.776353,1296.668677,1291.038565
2023-12-28 08:45:00+00:00,1302.58,15.06,0.000829,-0.039067,0.002650,0.027070,-0.024420,0.477149,-0.60,-0.038314,64.414781,0.385114,5.299511,-4.914397,1297.578111,1291.893486
2023-12-28 09:45:00+00:00,1302.78,15.12,0.000154,0.003976,0.002416,0.018568,-0.016152,0.281564,0.06,0.003984,64.598848,0.303197,5.375310,-5.072113,1298.378402,1292.699895
2023-12-29 03:45:00+00:00,1291.75,15.15,-0.008503,0.001982,0.003069,0.018516,-0.015446,0.233031,0.03,0.001984,49.417301,-0.516956,5.246071,-5.763027,1297.358648,1292.629532
2023-12-29 04:45:00+00:00,1295.48,15.03,0.002883,-0.007952,0.003127,0.018494,-0.015368,0.215987,-0.12,-0.007921,53.405222,-0.813699,5.042646,-5.856346,1297.069625,1292.840678
2023-12-29 05:45:00+00:00,1297.60,14.86,0.001635,-0.011375,0.003123,0.018684,-0.015561,0.196646,-0.17,-0.011311,55.550215,-0.867716,4.825717,-5.693434,1297.151221,1293.193220
2023-12-29 06:45:00+00:00,1296.93,14.60,-0.000516,-0.017652,0.003118,0.019091,-0.015973,0.202120,-0.26,-0.017497,54.693283,-0.942839,4.590007,-5.532847,1297.117187,1293.470019
2023-12-29 07:45:00+00:00,1292.78,14.51,-0.003205,-0.006183,0.003215,0.018966,-0.015751,0.212342,-0.09,-0.006164,49.590394,-1.247189,4.278210,-5.525399,1296.449928,1293.418906
2023-12-29 08:45:00+00:00,1291.35,14.48,-0.001107,-0.002070,0.003227,0.018965,-0.015738,0.213701,-0.03,-0.002068,47.930920,-1.502833,3.902502,-5.405335,1295.665323,1293.265654


In [6]:
from hmmlearn.hmm import GaussianHMM
from sklearn.preprocessing import StandardScaler

# Core features for regime inference
hmm_feature_cols = [
    "asset_log_ret",
    "vol_spread_20h",
    "asset_vix_corr_20h",
    "vix_velocity_1h",
]

hmm_df = features[hmm_feature_cols].dropna().copy()
if len(hmm_df) < 200:
    raise ValueError(
        f"Not enough rows for stable HMM fit: {len(hmm_df)} rows. "
        "Increase history or reduce feature NaNs."
    )

# Robustify against extreme spikes before scaling/modeling
for col in hmm_feature_cols:
    lower, upper = hmm_df[col].quantile([0.01, 0.99])
    hmm_df[col] = hmm_df[col].clip(lower=lower, upper=upper)

scaler = StandardScaler()
X = scaler.fit_transform(hmm_df.values)

n_states = 3
hmm_model = GaussianHMM(
    n_components=n_states,
    covariance_type="full",
    n_iter=500,
    random_state=42,
    min_covar=1e-6,
)
hmm_model.fit(X)

hidden_states = hmm_model.predict(X)
hmm_df["state"] = hidden_states

# Attach states back to main feature frame
features.loc[hmm_df.index, "hmm_state"] = hmm_df["state"]

# Transition matrix (state machine inertia)
transition_matrix = pd.DataFrame(
    hmm_model.transmat_,
    index=[f"state_{i}" for i in range(n_states)],
    columns=[f"state_{i}" for i in range(n_states)],
)

# Emission means in original (unscaled) feature space
emission_means_scaled = pd.DataFrame(hmm_model.means_, columns=hmm_feature_cols)
emission_means = pd.DataFrame(
    scaler.inverse_transform(emission_means_scaled),
    columns=hmm_feature_cols,
    index=[f"state_{i}" for i in range(n_states)],
)

# Heuristic state naming by average return (low->bear, mid->sideways, high->bull)
state_ret = hmm_df.groupby("state")["asset_log_ret"].mean().sort_values()
sorted_states = state_ret.index.tolist()
state_name_map = {
    sorted_states[0]: "Bear/High-Vol",
    sorted_states[1]: "Sideways/Chop",
    sorted_states[2]: "Bull/Low-Vol",
}
hmm_df["regime"] = hmm_df["state"].map(state_name_map)
features.loc[hmm_df.index, "regime"] = hmm_df["regime"]

state_summary = hmm_df.groupby(["state", "regime"])[hmm_feature_cols].mean().sort_index()
regime_counts = hmm_df["regime"].value_counts()

print(f"HMM trained on {len(hmm_df):,} rows with {n_states} states")
print("\nRegime counts:")
print(regime_counts)

print("\nTransition Matrix:")
display(transition_matrix)

print("\nEmission Means (original feature scale):")
display(emission_means)

print("\nState Summary (feature averages by state/regime):")
state_summary

HMM trained on 4,941 rows with 3 states

Regime counts:
regime
Sideways/Chop    1997
Bull/Low-Vol     1522
Bear/High-Vol    1422
Name: count, dtype: int64

Transition Matrix:


,state_0,state_1,state_2
state_0,0.953982,0.038422,0.007596
state_1,0.024909,0.949456,0.025634
state_2,0.010392,0.029852,0.959755



Emission Means (original feature scale):


,asset_log_ret,vol_spread_20h,asset_vix_corr_20h,vix_velocity_1h
state_0,-0.000087,-0.021345,-0.590415,0.019080
state_1,0.000003,-0.008225,-0.481029,-0.023523
state_2,0.000307,-0.010342,0.110027,0.003453



State Summary (feature averages by state/regime):


,,asset_log_ret,vol_spread_20h,asset_vix_corr_20h,vix_velocity_1h
state,regime,,,,
0,Bear/High-Vol,-0.000099,-0.021359,-0.592150,0.017820
1,Sideways/Chop,0.000006,-0.008166,-0.478644,-0.022829
2,Bull/Low-Vol,0.000317,-0.010415,0.113285,0.003909


## Regime Model: Gaussian Hidden Markov Model (HMM)
A Gaussian HMM models the market as a latent state machine with transition inertia.
- **States**: 3 latent regimes (Bull/Low-Vol, Sideways/Chop, Bear/High-Vol)
- **Transition matrix**: probability of staying/switching between regimes
- **Emission probabilities**: Gaussian likelihood of observed features per state

In [7]:
# Posterior probabilities from trained HMM
posterior = hmm_model.predict_proba(X)
posterior_cols = [f"p_state_{i}" for i in range(n_states)]
posterior_df = pd.DataFrame(posterior, index=hmm_df.index, columns=posterior_cols)

# Map state probabilities to semantic regime probabilities
regime_to_state = {name: state for state, name in state_name_map.items()}
bull_state = regime_to_state["Bull/Low-Vol"]
sideways_state = regime_to_state["Sideways/Chop"]
bear_state = regime_to_state["Bear/High-Vol"]

posterior_df["p_bull"] = posterior_df[f"p_state_{bull_state}"]
posterior_df["p_sideways"] = posterior_df[f"p_state_{sideways_state}"]
posterior_df["p_bear"] = posterior_df[f"p_state_{bear_state}"]

# Confidence percentages
posterior_df["bull_conf_pct"] = posterior_df["p_bull"] * 100
posterior_df["bear_conf_pct"] = posterior_df["p_bear"] * 100
posterior_df["max_conf_pct"] = posterior_df[["p_bull", "p_sideways", "p_bear"]].max(axis=1) * 100

# Hysteresis thresholds
BUY_THRESHOLD = 0.80
SELL_THRESHOLD = 0.70

# Enhanced action engine: HMM + Technical Indicators
actions = []
current_action = "HOLD"

for idx, row in posterior_df.iterrows():
    bull_p = row["p_bull"]
    bear_p = row["p_bear"]
    
    # Get technical indicator signals from features
    rsi = features.loc[idx, "rsi_14"] if idx in features.index else 50
    macd_hist = features.loc[idx, "macd_histogram"] if idx in features.index else 0
    ema_12 = features.loc[idx, "ema_12"] if idx in features.index else 0
    ema_26 = features.loc[idx, "ema_26"] if idx in features.index else 0
    close_price = features.loc[idx, "asset_close"] if idx in features.index else 0
    
    # Technical signal scoring
    tech_score = 0  # Range: -2 to +2
    
    # RSI signal: overbought (>70) = bearish, oversold (<30) = bullish
    if pd.notna(rsi):
        if rsi > 70:
            tech_score -= 1
        elif rsi < 30:
            tech_score += 1
    
    # MACD signal: positive histogram = bullish, negative = bearish
    if pd.notna(macd_hist):
        if macd_hist > 0:
            tech_score += 0.5
        else:
            tech_score -= 0.5
    
    # EMA signal: price above EMA12 above EMA26 = bullish
    if pd.notna(ema_12) and pd.notna(ema_26) and pd.notna(close_price):
        if close_price > ema_12 > ema_26:
            tech_score += 1
        elif close_price < ema_12 < ema_26:
            tech_score -= 1
    
    # Combined decision: HMM confidence + Technical confirmation
    if bull_p >= BUY_THRESHOLD and tech_score >= 0:
        current_action = "BUY"  # High bull confidence + positive or neutral tech
    elif bear_p >= SELL_THRESHOLD and tech_score <= 0:
        current_action = "SELL"  # High bear confidence + negative or neutral tech
    else:
        current_action = "HOLD"  # Hold otherwise (hysteresis buffer)

    actions.append(current_action)

posterior_df["action"] = actions

# Attach to main feature table
features.loc[posterior_df.index, "p_bull"] = posterior_df["p_bull"]
features.loc[posterior_df.index, "p_sideways"] = posterior_df["p_sideways"]
features.loc[posterior_df.index, "p_bear"] = posterior_df["p_bear"]
features.loc[posterior_df.index, "bull_conf_pct"] = posterior_df["bull_conf_pct"]
features.loc[posterior_df.index, "bear_conf_pct"] = posterior_df["bear_conf_pct"]
features.loc[posterior_df.index, "hmm_action"] = posterior_df["action"]

print("Action counts (with HMM + Technical Indicators):")
print(posterior_df["action"].value_counts())

posterior_df[[
    "p_bull",
    "p_sideways",
    "p_bear",
    "bull_conf_pct",
    "bear_conf_pct",
    "max_conf_pct",
    "action",
]].tail(15)

Action counts (with HMM + Technical Indicators):
action
HOLD    3495
BUY      779
SELL     667
Name: count, dtype: int64


,p_bull,p_sideways,p_bear,bull_conf_pct,bear_conf_pct,max_conf_pct,action
timestamp,,,,,,,
2023-12-27 09:45:00+00:00,1.000000,1.853036e-14,1.581599e-09,100.000000,1.581599e-07,100.000000,BUY
2023-12-28 03:45:00+00:00,1.000000,8.955198e-19,3.465854e-07,99.999965,3.465854e-05,99.999965,BUY
2023-12-28 04:45:00+00:00,1.000000,4.929286e-16,7.362974e-09,99.999999,7.362974e-07,99.999999,BUY
2023-12-28 05:45:00+00:00,1.000000,1.928245e-17,3.183616e-09,100.000000,3.183616e-07,100.000000,BUY
2023-12-28 06:45:00+00:00,1.000000,2.385666e-17,6.067089e-09,99.999999,6.067089e-07,99.999999,BUY
2023-12-28 07:45:00+00:00,1.000000,9.104127e-18,2.503185e-09,100.000000,2.503185e-07,100.000000,BUY
2023-12-28 08:45:00+00:00,1.000000,2.238346e-18,1.833671e-07,99.999982,1.833671e-05,99.999982,BUY
2023-12-28 09:45:00+00:00,1.000000,1.743608e-10,5.176276e-09,99.999999,5.176276e-07,99.999999,BUY
2023-12-29 03:45:00+00:00,1.000000,1.776975e-09,1.441275e-08,99.999998,1.441275e-06,99.999998,HOLD


In [8]:
# PnL report: first BUY signal to first subsequent SELL signal
signal_df = posterior_df.copy()
signal_df = signal_df.join(features[["asset_close"]], how="left")

buy_points = signal_df.index[signal_df["action"] == "BUY"]
if len(buy_points) == 0:
    raise ValueError("No BUY signal found in posterior_df['action'].")

buy_time = buy_points[0]
buy_price = float(signal_df.loc[buy_time, "asset_close"] )

sell_candidates = signal_df.loc[buy_time:].index[signal_df.loc[buy_time:, "action"] == "SELL"]
sell_candidates = sell_candidates[sell_candidates > buy_time]
if len(sell_candidates) == 0:
    raise ValueError("No SELL signal found after the first BUY signal.")

sell_time = sell_candidates[0]
sell_price = float(signal_df.loc[sell_time, "asset_close"] )

# 1-unit trade economics
absolute_pnl = sell_price - buy_price
pnl_pct = (sell_price / buy_price - 1.0) * 100
bars_held = int(signal_df.loc[buy_time:sell_time].shape[0] - 1)
hours_held = (sell_time - buy_time).total_seconds() / 3600.0

trade_report = pd.DataFrame(
    [
        {
            "entry_time": buy_time,
            "entry_price": buy_price,
            "exit_time": sell_time,
            "exit_price": sell_price,
            "bars_held": bars_held,
            "hours_held": round(hours_held, 2),
            "pnl_abs_per_1_unit": round(absolute_pnl, 4),
            "pnl_pct": round(pnl_pct, 4),
        }
    ]
)

print("First BUY -> first SELL trade report")
display(trade_report)

if pnl_pct >= 0:
    print(f"Result: PROFIT of {pnl_pct:.2f}% ({absolute_pnl:.4f} per unit)")
else:
    print(f"Result: LOSS of {pnl_pct:.2f}% ({absolute_pnl:.4f} per unit)")

First BUY -> first SELL trade report


,entry_time,entry_price,exit_time,exit_price,bars_held,hours_held,pnl_abs_per_1_unit,pnl_pct
0,2021-02-09 09:45:00+00:00,978.08,2021-02-19 07:45:00+00:00,1036.85,47,238.0,58.77,6.0087


Result: PROFIT of 6.01% (58.7700 per unit)


In [9]:
# Full backtest: all BUY->SELL trades across entire dataset
signal_df = posterior_df.copy()
signal_df = signal_df.join(features[["asset_close"]], how="left")

# Find all action transitions
all_trades = []
i = 0
while i < len(signal_df):
    # Find next BUY from position i
    buy_idx_mask = signal_df.iloc[i:].index[signal_df.iloc[i:]["action"] == "BUY"]
    if len(buy_idx_mask) == 0:
        break
    
    buy_time = buy_idx_mask[0]
    buy_pos = signal_df.index.get_loc(buy_time)
    buy_price = float(signal_df.loc[buy_time, "asset_close"])
    
    # Find next SELL after BUY
    sell_idx_mask = signal_df.iloc[buy_pos+1:].index[signal_df.iloc[buy_pos+1:]["action"] == "SELL"]
    if len(sell_idx_mask) == 0:
        break
    
    sell_time = sell_idx_mask[0]
    sell_pos = signal_df.index.get_loc(sell_time)
    sell_price = float(signal_df.loc[sell_time, "asset_close"])
    
    # Calculate trade metrics
    absolute_pnl_trade = sell_price - buy_price
    pnl_pct_trade = (sell_price / buy_price - 1.0) * 100
    bars_held_trade = sell_pos - buy_pos
    hours_held_trade = (sell_time - buy_time).total_seconds() / 3600.0
    
    all_trades.append({
        "entry_time": buy_time,
        "entry_price": buy_price,
        "exit_time": sell_time,
        "exit_price": sell_price,
        "bars_held": bars_held_trade,
        "hours_held": round(hours_held_trade, 2),
        "pnl_abs": round(absolute_pnl_trade, 4),
        "pnl_pct": round(pnl_pct_trade, 4),
    })
    
    # Move to next SELL to find next potential BUY
    i = sell_pos + 1

trades_df = pd.DataFrame(all_trades)

# Summary statistics
if len(trades_df) == 0:
    print("No complete trades found in dataset.")
else:
    total_trades = len(trades_df)
    winning_trades = (trades_df["pnl_pct"] > 0).sum()
    losing_trades = (trades_df["pnl_pct"] < 0).sum()
    breakeven_trades = (trades_df["pnl_pct"] == 0).sum()
    
    win_rate = (winning_trades / total_trades * 100) if total_trades > 0 else 0
    avg_win = trades_df[trades_df["pnl_pct"] > 0]["pnl_pct"].mean() if winning_trades > 0 else 0
    avg_loss = trades_df[trades_df["pnl_pct"] < 0]["pnl_pct"].mean() if losing_trades > 0 else 0
    avg_pnl_pct = trades_df["pnl_pct"].mean()
    total_pnl_pct = trades_df["pnl_pct"].sum()
    
    avg_bars = trades_df["bars_held"].mean()
    avg_hours = trades_df["hours_held"].mean()
    
    # Sharpe-like metric (pnl_pct series)
    pnl_std = trades_df["pnl_pct"].std()
    if pnl_std > 0:
        sharpe_approx = (avg_pnl_pct / pnl_std) * (252 ** 0.5)  # Annualized assuming 252 periods in year
    else:
        sharpe_approx = 0
    
    print("="*70)
    print("FULL DATASET BACKTEST REPORT".center(70))
    print("="*70)
    print(f"Total Trades:          {total_trades}")
    print(f"Winning Trades:        {winning_trades} ({win_rate:.1f}%)")
    print(f"Losing Trades:         {losing_trades}")
    print(f"Breakeven:             {breakeven_trades}")
    print(f"\nAverage Trade Metrics:")
    print(f"  Avg Win %:           {avg_win:.4f}%")
    print(f"  Avg Loss %:          {avg_loss:.4f}%")
    print(f"  Avg PnL per Trade:   {avg_pnl_pct:.4f}%")
    print(f"  Std Dev:             {pnl_std:.4f}%")
    print(f"  Sharpe (annualized): {sharpe_approx:.2f}")
    print(f"\nCumulative Results:")
    print(f"  Total PnL %:         {total_pnl_pct:.4f}%")
    print(f"  Avg Bars Held:       {avg_bars:.1f}h")
    print(f"  Avg Hours Held:      {avg_hours:.1f}h")
    print("="*70)
    
    print("\nTop 5 Trades (by PnL %):")
    top_trades = trades_df.nlargest(5, "pnl_pct")[["entry_time", "entry_price", "exit_time", "exit_price", "pnl_pct"]]
    display(top_trades)
    
    print("\nBottom 5 Trades (by PnL %):")
    bottom_trades = trades_df.nsmallest(5, "pnl_pct")[["entry_time", "entry_price", "exit_time", "exit_price", "pnl_pct"]]
    display(bottom_trades)
    
    print("\nAll Trades Table:")
    display(trades_df[["entry_time", "entry_price", "exit_time", "exit_price", "bars_held", "pnl_pct"]])

                     FULL DATASET BACKTEST REPORT                     
Total Trades:          24
Winning Trades:        10 (41.7%)
Losing Trades:         14
Breakeven:             0

Average Trade Metrics:
  Avg Win %:           6.1532%
  Avg Loss %:          -2.3681%
  Avg PnL per Trade:   1.1824%
  Std Dev:             7.0390%
  Sharpe (annualized): 2.67

Cumulative Results:
  Total PnL %:         28.3785%
  Avg Bars Held:       99.2h
  Avg Hours Held:      505.3h

Top 5 Trades (by PnL %):


,entry_time,entry_price,exit_time,exit_price,pnl_pct
4,2021-08-02 09:45:00+00:00,1036.25,2021-10-22 03:45:00+00:00,1317.18,27.1103
18,2023-03-27 03:45:00+00:00,1015.90,2023-05-29 04:45:00+00:00,1144.25,12.6341
19,2023-05-30 09:45:00+00:00,1143.88,2023-07-25 03:45:00+00:00,1246.43,8.9651
0,2021-02-09 09:45:00+00:00,978.08,2021-02-19 07:45:00+00:00,1036.85,6.0087
23,2023-10-30 09:45:00+00:00,1156.25,2023-11-28 03:45:00+00:00,1189.55,2.8800



Bottom 5 Trades (by PnL %):


,entry_time,entry_price,exit_time,exit_price,pnl_pct
6,2022-01-03 03:45:00+00:00,1193.83,2022-01-05 04:45:00+00:00,1106.16,-7.3436
21,2023-08-08 03:45:00+00:00,1264.25,2023-09-29 04:45:00+00:00,1174.00,-7.1386
5,2021-11-17 07:45:00+00:00,1237.50,2021-11-23 06:45:00+00:00,1186.40,-4.1293
22,2023-10-13 06:45:00+00:00,1174.48,2023-10-25 03:45:00+00:00,1138.13,-3.0950
8,2022-01-31 07:45:00+00:00,1090.34,2022-02-11 03:45:00+00:00,1066.53,-2.1837



All Trades Table:


,entry_time,entry_price,exit_time,exit_price,bars_held,pnl_pct
0,2021-02-09 09:45:00+00:00,978.08,2021-02-19 07:45:00+00:00,1036.85,47,6.0087
1,2021-04-26 03:45:00+00:00,977.53,2021-05-25 07:45:00+00:00,981.00,144,0.3550
2,2021-06-08 06:45:00+00:00,1107.50,2021-06-21 03:45:00+00:00,1106.78,60,-0.0650
3,2021-06-25 03:45:00+00:00,1047.20,2021-07-20 04:45:00+00:00,1044.80,120,-0.2292
4,2021-08-02 09:45:00+00:00,1036.25,2021-10-22 03:45:00+00:00,1317.18,386,27.1103
5,2021-11-17 07:45:00+00:00,1237.50,2021-11-23 06:45:00+00:00,1186.40,20,-4.1293
6,2022-01-03 03:45:00+00:00,1193.83,2022-01-05 04:45:00+00:00,1106.16,15,-7.3436
7,2022-01-11 09:45:00+00:00,1114.65,2022-01-21 03:45:00+00:00,1121.80,50,0.6415
8,2022-01-31 07:45:00+00:00,1090.34,2022-02-11 03:45:00+00:00,1066.53,52,-2.1837
9,2022-04-28 09:45:00+00:00,1280.02,2022-04-29 08:45:00+00:00,1262.72,6,-1.3515


In [10]:
# Buy-and-hold comparison
buy_hold_start = features["asset_close"].iloc[0]
buy_hold_end = features["asset_close"].iloc[-1]
buy_hold_return_pct = ((buy_hold_end / buy_hold_start) - 1.0) * 100

# Simple buy-and-hold has no trades, so duration is entire period
buy_hold_start_date = features.index[0]
buy_hold_end_date = features.index[-1]
buy_hold_duration_days = (buy_hold_end_date - buy_hold_start_date).days

# Active strategy stats (from previous backtest)
active_total_pnl = total_pnl_pct
active_trades = total_trades
active_avg_pnl = avg_pnl_pct
active_sharpe = sharpe_approx

# Simple Sharpe for buy-and-hold (treat daily return as 1 constant value)
daily_return_bh = (features["asset_close"].pct_change().mean()) * 252  # Annualized
daily_vol_bh = (features["asset_close"].pct_change().std()) * (252 ** 0.5)  # Annualized
if daily_vol_bh > 0:
    sharpe_bh = daily_return_bh / daily_vol_bh
else:
    sharpe_bh = 0

# Create comparison table
comparison = pd.DataFrame([
    {
        "Strategy": "Buy & Hold",
        "Total Return %": round(buy_hold_return_pct, 4),
        "Num Trades": 1,  # Single buy at start
        "Avg PnL per Trade %": round(buy_hold_return_pct, 4),
        "Win Rate %": 100 if buy_hold_return_pct > 0 else 0,
        "Sharpe (Annualized)": round(sharpe_bh, 2),
        "Duration (Days)": buy_hold_duration_days,
    },
    {
        "Strategy": "Active (HMM+RSI+MACD+EMA)",
        "Total Return %": round(active_total_pnl, 4),
        "Num Trades": active_trades,
        "Avg PnL per Trade %": round(active_avg_pnl, 4),
        "Win Rate %": round(win_rate, 1),
        "Sharpe (Annualized)": round(active_sharpe, 2),
        "Duration (Days)": buy_hold_duration_days,
    },
])

print("="*100)
print("STRATEGY COMPARISON: BUY-AND-HOLD vs ACTIVE TRADING".center(100))
print("="*100)
display(comparison)

# Calculate outperformance
outperformance = active_total_pnl - buy_hold_return_pct
trades_per_day = active_trades / buy_hold_duration_days if buy_hold_duration_days > 0 else 0

print(f"\nKey Insights:")
print(f"  Outperformance:     {outperformance:+.2f}% (Active beat Buy-Hold)")
print(f"  Trade Frequency:    {trades_per_day:.3f} trades/day")
print(f"  Active Trades Made: {active_trades} vs Buy-Hold's 1")
print(f"\n  Buy-Hold Entry:     {buy_hold_start_date.strftime('%Y-%m-%d %H:%M')} @ {buy_hold_start:.2f}")
print(f"  Buy-Hold Exit:      {buy_hold_end_date.strftime('%Y-%m-%d %H:%M')} @ {buy_hold_end:.2f}")
print(f"\n  Period: {buy_hold_start_date.strftime('%Y-%m-%d')} to {buy_hold_end_date.strftime('%Y-%m-%d')} ({buy_hold_duration_days} days)")
print("="*100)

                        STRATEGY COMPARISON: BUY-AND-HOLD vs ACTIVE TRADING                         


,Strategy,Total Return %,Num Trades,Avg PnL per Trade %,Win Rate %,Sharpe (Annualized),Duration (Days)
0,Buy & Hold,34.1930,1,34.1930,100.0,0.22,1057
1,Active (HMM+RSI+MACD+EMA),28.3785,24,1.1824,41.7,2.67,1057



Key Insights:
  Outperformance:     -5.81% (Active beat Buy-Hold)
  Trade Frequency:    0.023 trades/day
  Active Trades Made: 24 vs Buy-Hold's 1

  Buy-Hold Entry:     2021-02-05 03:45 @ 963.15
  Buy-Hold Exit:      2023-12-29 09:45 @ 1292.48

  Period: 2021-02-05 to 2023-12-29 (1057 days)


## Confidence Percent + Hysteresis Buffer
Use HMM posterior probabilities as confidence scores instead of hard class labels.
- **Raw posterior** at each hour: `[P(state_0), P(state_1), P(state_2)]`
- **Confidence %**: regime probability × 100
- **Hysteresis**:
  - Enter/keep **BUY** only when Bull confidence >= 80%
  - Enter/keep **SELL** only when Bear confidence >= 70%
  - Otherwise keep prior action (**HOLD buffer**) to avoid flicker

## Long-Term Labeling + Boosted Confidence + Explainability
This section adds:
- Triple Barrier labels (upper/lower/time barrier)
- Analyst features (SMA200 distance, ADX, ATR, Force Index)
- Boosted classifier confidence (`predict_proba`)
- SHAP explanation of feature contributions
- Walk-forward validation with early stopping where supported

In [11]:
import numpy as np
import pandas as pd
import ta

# {'epoch': 1703843100, 'open': 1291.1, 'high': 1294.5, 'low': 1290.6, 'close': 1292.48, 'volume': 1472108, 'timestamp_ist': '2023-12-29T15:15:00+05:30'},
# ---------- Build 1h OHLCV DataFrame from raw candles ----------
def candles_to_ohlcv_df(candles: list) -> pd.DataFrame:
    rows = []
    for candle in candles:
        if isinstance(candle, dict):
            ts = candle.get("epoch", candle.get("ts", candle.get("time", candle.get("datetime"))))
            open_ = candle.get("open", candle.get("o"))
            high = candle.get("high", candle.get("h"))
            low = candle.get("low", candle.get("l"))
            close = candle.get("close", candle.get("c"))
            volume = candle.get("volume", candle.get("v", 0.0))
        elif isinstance(candle, (list, tuple)) and len(candle) >= 5:
            ts = candle[0]
            open_ = candle[1]
            high = candle[2]
            low = candle[3]
            close = candle[4]
            volume = candle[5] if len(candle) > 5 else 0.0
        else:
            continue

        rows.append(
            {
                "timestamp": ts,
                "open": open_,
                "high": high,
                "low": low,
                "close": close,
                "volume": volume,
            }
        )

    ohlcv = pd.DataFrame(rows)
    if ohlcv.empty:
        raise ValueError("No parseable OHLCV rows found in ohlcv_data.")

    ts = ohlcv["timestamp"]
    if pd.api.types.is_numeric_dtype(ts):
        idx = pd.to_datetime(ts, unit="s", utc=True, errors="coerce")
        if idx.isna().all():
            idx = pd.to_datetime(ts, unit="ms", utc=True, errors="coerce")
    else:
        idx = pd.to_datetime(ts, utc=True, errors="coerce")

    ohlcv["timestamp"] = idx
    for col in ["open", "high", "low", "close", "volume"]:
        ohlcv[col] = pd.to_numeric(ohlcv[col], errors="coerce")

    ohlcv = (
        ohlcv.dropna(subset=["timestamp", "open", "high", "low", "close"])
        .drop_duplicates(subset=["timestamp"], keep="last")
        .sort_values("timestamp")
        .set_index("timestamp")
    )
    return ohlcv

asset_ohlcv = candles_to_ohlcv_df(ohlcv_data)
print(f"OHLCV rows: {len(asset_ohlcv):,} | Range: {asset_ohlcv.index.min()} -> {asset_ohlcv.index.max()}")
asset_ohlcv.tail(5)
# print(ohlcv_data)

OHLCV rows: 5,000 | Range: 2021-02-05 03:45:00+00:00 -> 2023-12-29 09:45:00+00:00


,open,high,low,close,volume
timestamp,,,,,
2023-12-29 05:45:00+00:00,1295.38,1298.90,1295.35,1297.60,1399918
2023-12-29 06:45:00+00:00,1297.60,1298.40,1294.00,1296.93,1209704
2023-12-29 07:45:00+00:00,1296.90,1297.68,1291.25,1292.78,1534988
2023-12-29 08:45:00+00:00,1292.65,1294.45,1290.48,1291.35,2371328
2023-12-29 09:45:00+00:00,1291.10,1294.50,1290.60,1292.48,1472108


In [12]:
# ---------- Triple Barrier Labels ----------
# Upper Barrier (PT): +5%, Lower Barrier (SL): -2%, Time Barrier: 20 hours
PT_PCT = 0.05
SL_PCT = 0.02
TIME_BARRIER_H = 20

def triple_barrier_labels(
    df: pd.DataFrame,
    pt_pct: float = PT_PCT,
    sl_pct: float = SL_PCT,
    time_h: int = TIME_BARRIER_H,
    price_col: str = "close",
    high_col: str = "high",
    low_col: str = "low",
) -> pd.DataFrame:
    out = df[[price_col, high_col, low_col]].copy()
    n = len(out)
    labels = np.full(n, np.nan)
    event_end = [pd.NaT] * n
    barrier_hit = [None] * n

    closes = out[price_col].to_numpy()
    highs = out[high_col].to_numpy()
    lows = out[low_col].to_numpy()
    idx = out.index

    for i in range(n - time_h):
        entry = closes[i]
        up = entry * (1 + pt_pct)
        dn = entry * (1 - sl_pct)

        y = 0  # 0 = time barrier first
        hit_name = "time"
        end_i = i + time_h

        for j in range(i + 1, i + time_h + 1):
            hit_up = highs[j] >= up
            hit_dn = lows[j] <= dn

            if hit_up and hit_dn:
                if abs(highs[j] - up) <= abs(lows[j] - dn):
                    y = 1
                    hit_name = "upper"
                else:
                    y = -1
                    hit_name = "lower"
                end_i = j
                break
            if hit_up:
                y = 1
                hit_name = "upper"
                end_i = j
                break
            if hit_dn:
                y = -1
                hit_name = "lower"
                end_i = j
                break

        labels[i] = y
        barrier_hit[i] = hit_name
        event_end[i] = idx[end_i]

    out["tb_label"] = labels
    out["tb_barrier_hit"] = barrier_hit
    out["tb_event_end"] = event_end
    out = out.dropna(subset=["tb_label"]).copy()
    out["tb_label"] = out["tb_label"].astype(int)
    return out

tb_df = triple_barrier_labels(asset_ohlcv)
print("Triple-barrier label distribution (1=upper, 0=time, -1=lower):")
print(tb_df["tb_label"].value_counts().sort_index())
tb_df[["close", "tb_label", "tb_barrier_hit", "tb_event_end"]].head(10)

Triple-barrier label distribution (1=upper, 0=time, -1=lower):
tb_label
-1    1709
 0    2999
 1     272
Name: count, dtype: int64


,close,tb_label,tb_barrier_hit,tb_event_end
timestamp,,,,
2021-02-05 03:45:00+00:00,963.15,0,time,2021-02-09 09:45:00+00:00
2021-02-05 04:45:00+00:00,965.53,0,time,2021-02-10 03:45:00+00:00
2021-02-05 05:45:00+00:00,964.50,0,time,2021-02-10 04:45:00+00:00
2021-02-05 06:45:00+00:00,963.50,0,time,2021-02-10 05:45:00+00:00
2021-02-05 07:45:00+00:00,965.33,0,time,2021-02-10 06:45:00+00:00
2021-02-05 08:45:00+00:00,962.38,0,time,2021-02-10 07:45:00+00:00
2021-02-05 09:45:00+00:00,962.48,0,time,2021-02-10 08:45:00+00:00
2021-02-08 03:45:00+00:00,974.85,0,time,2021-02-10 09:45:00+00:00
2021-02-08 04:45:00+00:00,974.85,0,time,2021-02-11 03:45:00+00:00


In [13]:
# ---------- Analyst Features (trend, momentum, volatility, volume) ----------
model_df = asset_ohlcv.copy()

# Trend
model_df["sma_200"] = model_df["close"].rolling(200).mean()
model_df["dist_sma_200"] = (model_df["close"] / model_df["sma_200"]) - 1.0

# Momentum
model_df["adx_14"] = ta.trend.adx(model_df["high"], model_df["low"], model_df["close"], window=14)

# Volatility
model_df["atr_14"] = ta.volatility.average_true_range(
    model_df["high"], model_df["low"], model_df["close"], window=14
)
model_df["atr_pct_14"] = model_df["atr_14"] / model_df["close"]

# Volume
model_df["force_index_13"] = ta.volume.force_index(
    model_df["close"], model_df["volume"], window=13
)

# Extra context features
model_df["ret_1h"] = model_df["close"].pct_change()
model_df["ret_6h"] = model_df["close"].pct_change(6)
model_df["ret_24h"] = model_df["close"].pct_change(24)
model_df["rsi_14"] = ta.momentum.rsi(model_df["close"], window=14)

# Join labels
model_df = model_df.join(tb_df[["tb_label", "tb_barrier_hit", "tb_event_end"]], how="inner")

feature_cols_long = [
    "dist_sma_200",
    "adx_14",
    "atr_14",
    "atr_pct_14",
    "force_index_13",
    "ret_1h",
    "ret_6h",
    "ret_24h",
    "rsi_14",
]

model_df = model_df.dropna(subset=feature_cols_long + ["tb_label"]).copy()
X_long = model_df[feature_cols_long].copy()
y_long = model_df["tb_label"].copy()

print(f"Model rows: {len(model_df):,}")
print("Label balance:")
print(y_long.value_counts().sort_index())
X_long.tail(5)

Model rows: 4,781
Label balance:
tb_label
-1    1617
 0    2932
 1     232
Name: count, dtype: int64


/home/windy/miniconda3/envs/trade/lib/python3.11/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in scalar divide
  dip[idx] = 100 * (self._dip[idx] / value)
/home/windy/miniconda3/envs/trade/lib/python3.11/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in scalar divide
  din[idx] = 100 * (self._din[idx] / value)


,dist_sma_200,adx_14,atr_14,atr_pct_14,force_index_13,ret_1h,ret_6h,ret_24h,rsi_14
timestamp,,,,,,,,,
2023-12-26 06:45:00+00:00,0.059328,36.040077,8.154775,0.006320,3.406495e+06,-0.001354,0.007119,-0.000836,61.513038
2023-12-26 07:45:00+00:00,0.057879,36.450499,7.770148,0.006027,2.798920e+06,-0.000829,0.005028,-0.006910,60.129096
2023-12-26 08:45:00+00:00,0.057349,36.762823,7.411566,0.005749,2.408471e+06,0.000039,0.005224,-0.002453,60.174186
2023-12-26 09:45:00+00:00,0.056619,36.923156,7.016454,0.005443,2.043032e+06,-0.000155,-0.000364,-0.002067,59.882461
2023-12-27 03:45:00+00:00,0.058630,37.726827,7.122422,0.005512,2.404712e+06,0.002459,-0.000966,0.019342,62.948542


In [14]:
# ---------- Boosted model confidence (predict which barrier hits first) ----------
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split

# Prefer XGBoost -> LightGBM -> sklearn fallback
model_name = None
booster = None
lgb = None

try:
    from xgboost import XGBClassifier
    model_name = "xgboost"
except Exception:
    XGBClassifier = None

try:
    import lightgbm as lgb
    from lightgbm import LGBMClassifier
    if model_name is None:
        model_name = "lightgbm"
except Exception:
    LGBMClassifier = None

if model_name == "xgboost":
    booster = XGBClassifier(
        n_estimators=500,
        learning_rate=0.03,
        max_depth=5,
        subsample=0.9,
        colsample_bytree=0.9,
        objective="multi:softprob",
        num_class=3,
        random_state=42,
        eval_metric="mlogloss",
    )
elif model_name == "lightgbm":
    booster = LGBMClassifier(
        n_estimators=500,
        learning_rate=0.03,
        max_depth=-1,
        num_leaves=31,
        subsample=0.9,
        colsample_bytree=0.9,
        random_state=42,
        objective="multiclass",
        num_class=3,
    )
else:
    from sklearn.ensemble import GradientBoostingClassifier
    model_name = "sklearn_gb_fallback"
    booster = GradientBoostingClassifier(random_state=42)

# map labels from {-1,0,1} -> {0,1,2} for multiclass tools
label_map = {-1: 0, 0: 1, 1: 2}
inv_label_map = {v: k for k, v in label_map.items()}
y_encoded = y_long.map(label_map).astype(int)

# time-aware split
split_idx = int(len(X_long) * 0.8)
X_train, X_test = X_long.iloc[:split_idx], X_long.iloc[split_idx:]
y_train, y_test = y_encoded.iloc[:split_idx], y_encoded.iloc[split_idx:]

# early stopping where supported
if model_name == "xgboost":
    try:
        booster.fit(
            X_train,
            y_train,
            eval_set=[(X_test, y_test)],
            early_stopping_rounds=50,
            verbose=False,
        )
    except TypeError:
        booster.fit(
            X_train,
            y_train,
            eval_set=[(X_test, y_test)],
            verbose=False,
        )
elif model_name == "lightgbm":
    try:
        booster.fit(
            X_train,
            y_train,
            eval_set=[(X_test, y_test)],
            callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=False)],
        )
    except Exception:
        booster.fit(X_train, y_train)
else:
    booster.fit(X_train, y_train)

proba = booster.predict_proba(X_test)
pred = np.argmax(proba, axis=1)
pred_label = pd.Series(pred, index=X_test.index).map(inv_label_map)

proba_df = pd.DataFrame(
    proba,
    index=X_test.index,
    columns=["p_lower_first", "p_time_first", "p_upper_first"],
)
proba_df["pred_tb_label"] = pred_label
proba_df["pred_conf_pct"] = proba_df[["p_lower_first", "p_time_first", "p_upper_first"]].max(axis=1) * 100
proba_df["true_tb_label"] = y_test.map(inv_label_map)

print(f"Model used: {model_name}")
print(f"Accuracy: {accuracy_score(y_test, pred):.4f}")
print("\nClassification report (encoded classes 0/1/2 => -1/0/1):")
print(classification_report(y_test, pred, zero_division=0))

print("\nTop confidence predictions:")
display(proba_df.sort_values("pred_conf_pct", ascending=False).head(10))

/home/windy/miniconda3/envs/trade/lib/python3.11/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


Model used: xgboost
Accuracy: 0.7137

Classification report (encoded classes 0/1/2 => -1/0/1):
              precision    recall  f1-score   support

           0       0.31      0.23      0.26       200
           1       0.79      0.86      0.82       739
           2       0.00      0.00      0.00        18

    accuracy                           0.71       957
   macro avg       0.37      0.36      0.36       957
weighted avg       0.67      0.71      0.69       957


Top confidence predictions:


,p_lower_first,p_time_first,p_upper_first,pred_tb_label,pred_conf_pct,true_tb_label
timestamp,,,,,,
2023-09-08 04:45:00+00:00,0.038388,0.929831,0.031781,0,92.983109,0
2023-09-07 05:45:00+00:00,0.042102,0.928011,0.029887,0,92.801102,0
2023-10-11 04:45:00+00:00,0.042801,0.927758,0.029441,0,92.775749,0
2023-11-22 09:45:00+00:00,0.050085,0.927750,0.022165,0,92.774994,0
2023-06-27 04:45:00+00:00,0.048676,0.926871,0.024453,0,92.687080,0
2023-10-10 05:45:00+00:00,0.036036,0.925884,0.038080,0,92.588409,0
2023-10-06 09:45:00+00:00,0.037349,0.925614,0.037037,0,92.561432,0
2023-10-17 05:45:00+00:00,0.040961,0.925133,0.033906,0,92.513260,-1
2023-10-10 04:45:00+00:00,0.037786,0.924939,0.037275,0,92.493904,0


In [15]:
# ---------- SHAP feature analysis (why the model is confident) ----------
try:
    import shap

    # Sample for speed
    sample_n = min(400, len(X_test))
    X_shap = X_test.sample(sample_n, random_state=42) if len(X_test) > sample_n else X_test.copy()

    explainer = shap.TreeExplainer(booster)
    shap_values = explainer.shap_values(X_shap)

    # Multiclass support: shap_values is often a list with one array per class
    if isinstance(shap_values, list):
        # class index 2 corresponds to tb_label=+1 (upper barrier first) after mapping
        class_idx = 2 if len(shap_values) > 2 else 0
        mean_abs_shap = np.abs(shap_values[class_idx]).mean(axis=0)
    else:
        mean_abs_shap = np.abs(shap_values).mean(axis=0)

    shap_importance = pd.DataFrame(
        {"feature": X_shap.columns, "mean_abs_shap": mean_abs_shap}
    ).sort_values("mean_abs_shap", ascending=False)

    print("Top SHAP features (global importance):")
    display(shap_importance.head(15))

    # Optional plot (works in notebook UI when JS available)
    try:
        shap.summary_plot(shap_values, X_shap, show=False)
    except Exception:
        pass

except Exception as shap_err:
    print("SHAP not available or failed for this model/environment.")
    print(f"Reason: {shap_err}")
    print("Install with: pip install shap")

/home/windy/miniconda3/envs/trade/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


SHAP not available or failed for this model/environment.
Reason: Per-column arrays must each be 1-dimensional
Install with: pip install shap


In [16]:
# ---------- Walk-forward validation + early stopping ----------
from sklearn.metrics import f1_score

def build_booster(name: str):
    if name == "xgboost" and XGBClassifier is not None:
        return XGBClassifier(
            n_estimators=1000,
            learning_rate=0.03,
            max_depth=5,
            subsample=0.9,
            colsample_bytree=0.9,
            objective="multi:softprob",
            num_class=3,
            random_state=42,
            eval_metric="mlogloss",
        )
    if name == "lightgbm" and LGBMClassifier is not None:
        return LGBMClassifier(
            n_estimators=1000,
            learning_rate=0.03,
            num_leaves=31,
            subsample=0.9,
            colsample_bytree=0.9,
            objective="multiclass",
            num_class=3,
            random_state=42,
        )
    from sklearn.ensemble import GradientBoostingClassifier
    return GradientBoostingClassifier(random_state=42)

# Rolling: train 6 months, test next 1 month (time ordered)
wf_df = X_long.copy()
wf_df["y"] = y_encoded.values
wf_df = wf_df.sort_index().copy()
wf_df["year_month"] = wf_df.index.to_period("M")
months = sorted(wf_df["year_month"].unique())

train_months = 6
test_months = 1
wf_rows = []

for start in range(0, len(months) - (train_months + test_months) + 1):
    tr_slice = months[start : start + train_months]
    te_slice = months[start + train_months : start + train_months + test_months]

    tr_mask = wf_df["year_month"].isin(tr_slice)
    te_mask = wf_df["year_month"].isin(te_slice)

    train_data = wf_df.loc[tr_mask]
    test_data = wf_df.loc[te_mask]

    if len(train_data) < 300 or len(test_data) < 30:
        continue

    X_tr = train_data[feature_cols_long]
    y_tr = train_data["y"].astype(int)
    X_te = test_data[feature_cols_long]
    y_te = test_data["y"].astype(int)

    wf_model = build_booster(model_name)

    if model_name == "xgboost":
        try:
            wf_model.fit(
                X_tr,
                y_tr,
                eval_set=[(X_te, y_te)],
                early_stopping_rounds=50,
                verbose=False,
            )
        except TypeError:
            wf_model.fit(X_tr, y_tr, eval_set=[(X_te, y_te)], verbose=False)
    elif model_name == "lightgbm":
        try:
            wf_model.fit(
                X_tr,
                y_tr,
                eval_set=[(X_te, y_te)],
                callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=False)],
            )
        except Exception:
            wf_model.fit(X_tr, y_tr)
    else:
        wf_model.fit(X_tr, y_tr)

    y_hat = wf_model.predict(X_te)
    f1m = f1_score(y_te, y_hat, average="macro")
    acc = accuracy_score(y_te, y_hat)

    wf_rows.append(
        {
            "train_start": str(tr_slice[0]),
            "train_end": str(tr_slice[-1]),
            "test_month": str(te_slice[0]),
            "test_n": int(len(X_te)),
            "acc": round(float(acc), 4),
            "f1_macro": round(float(f1m), 4),
        }
    )

wf_results = pd.DataFrame(wf_rows)
if wf_results.empty:
    print("No walk-forward windows were created. Consider larger date coverage.")
else:
    print("Walk-forward results (6M train -> 1M test):")
    display(wf_results)
    print("\nAverages:")
    print(wf_results[["acc", "f1_macro"]].mean().round(4))

/tmp/ipykernel_1679431/18112445.py:35: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  wf_df["year_month"] = wf_df.index.to_period("M")
/home/windy/miniconda3/envs/trade/lib/python3.11/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/home/windy/miniconda3/envs/trade/lib/python3.11/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/home/windy/miniconda3/envs/trade/lib/python3.11/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params

Walk-forward results (6M train -> 1M test):


,train_start,train_end,test_month,test_n,acc,f1_macro
0,2021-03,2021-08,2021-09,147,0.6667,0.3963
1,2021-04,2021-09,2021-10,140,0.4643,0.2203
2,2021-05,2021-10,2021-11,135,0.4667,0.2980
3,2021-06,2021-11,2021-12,161,0.5839,0.3592
4,2021-07,2021-12,2022-01,140,0.5071,0.3516
5,2021-08,2022-01,2022-02,133,0.3985,0.2658
6,2021-09,2022-02,2022-03,147,0.4762,0.3511
7,2021-10,2022-03,2022-04,133,0.3459,0.3540
8,2021-11,2022-04,2022-05,147,0.3129,0.2226
9,2021-12,2022-05,2022-06,154,0.5519,0.4581



Averages:
acc         0.5591
f1_macro    0.3775
dtype: float64


In [17]:

# ---------- HMM-only vs HMM + Technical Indicators ----------
# Rebuild HMM-only actions using the same hysteresis thresholds but NO technical filter

hmm_only_actions = []
for idx_row, row in posterior_df.iterrows():
    bull_p = row["p_bull"]
    bear_p = row["p_bear"]
    if bull_p >= BUY_THRESHOLD:
        action = "BUY"
    elif bear_p >= SELL_THRESHOLD:
        action = "SELL"
    else:
        action = "HOLD"
    hmm_only_actions.append(action)

hmm_only_df = posterior_df.copy()
hmm_only_df["action_hmm_only"] = hmm_only_actions

print("HMM-only action counts:")
print(hmm_only_df["action_hmm_only"].value_counts())
print("\nHMM + Tech action counts:")
print(posterior_df["action"].value_counts())


def backtest_signals(signal_series: pd.Series, price_series: pd.Series) -> dict:
    """Run a simple long-only BUY->SELL backtest and return summary stats."""
    sig_df = pd.DataFrame({"action": signal_series, "price": price_series}).dropna()
    trades = []
    i = 0
    idx_vals = sig_df.index
    actions_arr = sig_df["action"].values
    prices_arr = sig_df["price"].values

    while i < len(sig_df):
        # Find next BUY
        buy_pos = next((j for j in range(i, len(sig_df)) if actions_arr[j] == "BUY"), None)
        if buy_pos is None:
            break
        buy_price_val = prices_arr[buy_pos]

        # Find next SELL after BUY
        sell_pos = next((j for j in range(buy_pos + 1, len(sig_df)) if actions_arr[j] == "SELL"), None)
        if sell_pos is None:
            break
        sell_price_val = prices_arr[sell_pos]

        pnl_pct_val = (sell_price_val / buy_price_val - 1.0) * 100
        hours = (idx_vals[sell_pos] - idx_vals[buy_pos]).total_seconds() / 3600.0
        trades.append({
            "entry_time": idx_vals[buy_pos],
            "entry_price": buy_price_val,
            "exit_time": idx_vals[sell_pos],
            "exit_price": sell_price_val,
            "bars_held": sell_pos - buy_pos,
            "hours_held": round(hours, 2),
            "pnl_pct": round(pnl_pct_val, 4),
        })
        i = sell_pos + 1

    if not trades:
        return {"n_trades": 0, "total_pnl": 0, "win_rate": 0, "avg_pnl": 0, "sharpe": 0, "trades_df": pd.DataFrame()}

    df = pd.DataFrame(trades)
    n = len(df)
    wins = (df["pnl_pct"] > 0).sum()
    avg = df["pnl_pct"].mean()
    std = df["pnl_pct"].std()
    sharpe = (avg / std) * (252 ** 0.5) if std > 0 else 0

    return {
        "n_trades": n,
        "total_pnl": round(df["pnl_pct"].sum(), 4),
        "win_rate": round(wins / n * 100, 2),
        "avg_pnl": round(avg, 4),
        "sharpe": round(sharpe, 4),
        "trades_df": df,
    }


# Price series aligned to posterior_df index
price_aligned = signal_df["asset_close"].reindex(posterior_df.index)

stats_hmm_only = backtest_signals(hmm_only_df["action_hmm_only"], price_aligned)
stats_hmm_tech = backtest_signals(posterior_df["action"], price_aligned)

# Buy-and-hold reference (already computed)
comparison2 = pd.DataFrame([
    {
        "Strategy": "HMM Only",
        "Total Return %": stats_hmm_only["total_pnl"],
        "Num Trades": stats_hmm_only["n_trades"],
        "Win Rate %": stats_hmm_only["win_rate"],
        "Avg PnL per Trade %": stats_hmm_only["avg_pnl"],
        "Sharpe (Annualized)": stats_hmm_only["sharpe"],
    },
    {
        "Strategy": "HMM + RSI + MACD + EMA",
        "Total Return %": stats_hmm_tech["total_pnl"],
        "Num Trades": stats_hmm_tech["n_trades"],
        "Win Rate %": stats_hmm_tech["win_rate"],
        "Avg PnL per Trade %": stats_hmm_tech["avg_pnl"],
        "Sharpe (Annualized)": stats_hmm_tech["sharpe"],
    },
    {
        "Strategy": "Buy & Hold",
        "Total Return %": round(buy_hold_return_pct, 4),
        "Num Trades": 1,
        "Win Rate %": 100 if buy_hold_return_pct > 0 else 0,
        "Avg PnL per Trade %": round(buy_hold_return_pct, 4),
        "Sharpe (Annualized)": round(sharpe_bh, 4),
    },
])

print("=" * 90)
print("HMM-ONLY vs HMM + TECH INDICATORS vs BUY-AND-HOLD".center(90))
print("=" * 90)
display(comparison2)

delta = stats_hmm_tech["total_pnl"] - stats_hmm_only["total_pnl"]
delta_sharpe = stats_hmm_tech["sharpe"] - stats_hmm_only["sharpe"]
delta_wr = stats_hmm_tech["win_rate"] - stats_hmm_only["win_rate"]
print(f"\nTech overlay adds: {delta:+.2f}% total return | "
      f"Sharpe {delta_sharpe:+.4f} | "
      f"Win rate {delta_wr:+.2f}% | "
      f"Trade count diff: {stats_hmm_tech['n_trades'] - stats_hmm_only['n_trades']:+d}")


HMM-only action counts:
action_hmm_only
HOLD    2136
BUY     1459
SELL    1346
Name: count, dtype: int64

HMM + Tech action counts:
action
HOLD    3495
BUY      779
SELL     667
Name: count, dtype: int64
                    HMM-ONLY vs HMM + TECH INDICATORS vs BUY-AND-HOLD                     


,Strategy,Total Return %,Num Trades,Win Rate %,Avg PnL per Trade %,Sharpe (Annualized)
0,HMM Only,18.6021,26,42.31,0.7155,1.6880
1,HMM + RSI + MACD + EMA,28.3785,24,41.67,1.1824,2.6666
2,Buy & Hold,34.1930,1,100.00,34.1930,0.2165



Tech overlay adds: +9.78% total return | Sharpe +0.9786 | Win rate -0.64% | Trade count diff: -2


In [18]:
# ---------- Export trained models ----------
import json
import pathlib
import joblib

# Output directory (sits next to the notebook)
MODEL_DIR = pathlib.Path("models") / f"{ticker}_{timeframe}"
MODEL_DIR.mkdir(parents=True, exist_ok=True)

# 1) HMM + its scaler  (hmmlearn objects are joblib-serialisable)
joblib.dump(hmm_model, MODEL_DIR / "hmm_model.joblib")
joblib.dump(scaler, MODEL_DIR / "hmm_scaler.joblib")

# 2) Boosted classifier (XGBoost native JSON keeps exact tree structure)
if model_name == "xgboost":
    booster.save_model(str(MODEL_DIR / "booster_xgb.json"))
elif model_name == "lightgbm":
    booster.booster_.save_model(str(MODEL_DIR / "booster_lgb.txt"))
else:
    joblib.dump(booster, MODEL_DIR / "booster_sklearn.joblib")

# 3) Metadata: feature lists, label maps, thresholds, state names
metadata = {
    "ticker": ticker,
    "timeframe": timeframe,
    "model_name": model_name,
    "hmm_feature_cols": hmm_feature_cols,
    "feature_cols_long": feature_cols_long,
    "n_states": n_states,
    "state_name_map": {int(k): v for k, v in state_name_map.items()},
    "label_map": {int(k): int(v) for k, v in label_map.items()},
    "inv_label_map": {int(k): int(v) for k, v in inv_label_map.items()},
    "buy_threshold": BUY_THRESHOLD,
    "sell_threshold": SELL_THRESHOLD,
    "pt_pct": PT_PCT,
    "sl_pct": SL_PCT,
    "time_barrier_h": TIME_BARRIER_H,
    "train_start": str(X_long.index.min().date()),
    "train_end": str(X_long.index.max().date()),
}
with open(MODEL_DIR / "metadata.json", "w") as f:
    json.dump(metadata, f, indent=2)

# 4) Summary report
print(f"Models saved to: {MODEL_DIR.resolve()}/")
for p in sorted(MODEL_DIR.iterdir()):
    size_kb = p.stat().st_size / 1024
    print(f"  {p.name:<35} {size_kb:>8.1f} KB")

print(f"\nTo reload:")
print(f"  hmm  = joblib.load('{MODEL_DIR}/hmm_model.joblib')")
print(f"  scaler = joblib.load('{MODEL_DIR}/hmm_scaler.joblib')")
if model_name == "xgboost":
    print(f"  from xgboost import XGBClassifier")
    print(f"  booster = XGBClassifier(); booster.load_model('{MODEL_DIR}/booster_xgb.json')")
elif model_name == "lightgbm":
    print(f"  import lightgbm as lgb")
    print(f"  booster = lgb.Booster(model_file='{MODEL_DIR}/booster_lgb.txt')")
else:
    print(f"  booster = joblib.load('{MODEL_DIR}/booster_sklearn.joblib')")


Models saved to: /mnt/drive3/500GB/p1/Projects/trading-strats/experiments/models/RELIANCE_1h/
  booster_xgb.json                      1677.8 KB
  hmm_model.joblib                         2.3 KB
  hmm_scaler.joblib                        0.7 KB
  metadata.json                            0.8 KB

To reload:
  hmm  = joblib.load('models/RELIANCE_1h/hmm_model.joblib')
  scaler = joblib.load('models/RELIANCE_1h/hmm_scaler.joblib')
  from xgboost import XGBClassifier
  booster = XGBClassifier(); booster.load_model('models/RELIANCE_1h/booster_xgb.json')
